# Import and Install Dependencies

In [ ]:
!pip install  matplotlib

# Clone data

In [ ]:
!git clone https://github.com/NgKhTr/CS114_N21_2Q2T.git
%cd content/CS114_N21_2Q2T

# Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from creatmodel import create_model
import os
import numpy as np
from init import parameter
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,f1_score


In [ ]:
actions=parameter["actions"]
label_map = {label:num for num, label in enumerate(actions)}
no_sequences=parameter["no_sequences"]
FPS=parameter["FPS"]
DATA_PATH=parameter["DATA_PATH"]
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(FPS):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
X.shape

# Train LSTM Neural Network

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model=create_model(actions)
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
print("ACC:",accuracy_score(ytrue, yhat))
print("f1:", f1_score(ytrue, yhat))

In [ ]:
model.save('action.h5')